In [40]:
import pandas as pd

In [41]:
#transactions.csv too large for github, contact me if you require the csvdata = pd.read_csv('transactions.csv')

# Preprocess Data

In [42]:
#We only want transfers and cash_outs
X = data.loc[(data.type == 'TRANSFER') | (data.type == 'CASH_OUT')]

In [43]:
X

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.00,0.0,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.0,C38997010,21182.00,0.00,1,0
15,1,CASH_OUT,229133.94,C905080434,15325.00,0.0,C476402209,5083.00,51513.44,0,0
19,1,TRANSFER,215310.30,C1670993182,705.00,0.0,C1100439041,22425.00,0.00,0,0
24,1,TRANSFER,311685.89,C1984094095,10835.00,0.0,C932583850,6267.00,2719172.89,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1,0


In [44]:
#remove meaningless columns
X = X.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)

In [45]:
X['type'] = X['type'].replace({'TRANSFER':0,'CASH_OUT':1})

In [46]:
#remove all accounts where there is no money
#results in higher model accuracy
X.drop(X.loc[(X.oldbalanceDest == 0) & (X.newbalanceDest == 0) & (X.amount == 0)], axis=1);
X.drop(X.loc[(X.oldbalanceOrg == 0) & (X.newbalanceOrig == 0) & (X.amount == 0)], axis=1);

In [47]:
y = X['isFraud']

In [48]:
del X['isFraud']

In [49]:
X

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
2,1,0,181.00,181.00,0.0,0.00,0.00
3,1,1,181.00,181.00,0.0,21182.00,0.00
15,1,1,229133.94,15325.00,0.0,5083.00,51513.44
19,1,0,215310.30,705.00,0.0,22425.00,0.00
24,1,0,311685.89,10835.00,0.0,6267.00,2719172.89
...,...,...,...,...,...,...,...
6362615,743,1,339682.13,339682.13,0.0,0.00,339682.13
6362616,743,0,6311409.28,6311409.28,0.0,0.00,0.00
6362617,743,1,6311409.28,6311409.28,0.0,68488.84,6379898.11
6362618,743,0,850002.52,850002.52,0.0,0.00,0.00


In [50]:
y

2          1
3          1
15         0
19         0
24         0
          ..
6362615    1
6362616    1
6362617    1
6362618    1
6362619    1
Name: isFraud, Length: 2770409, dtype: int64

# Train Model

In [51]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [53]:
#Create model and configure it to account for imbalanced dataset
weights = (y==0).sum() / (1.0 * (y==1).sum())

In [54]:
xgb = XGBClassifier(scale_pos_weight = weights, n_jobs = 4)

In [55]:
xgb.fit(X_train, y_train.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=4,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=336.3199805186899,
              seed=None, silent=None, subsample=1, verbosity=1)

# Test Model

In [56]:
proba = xgb.predict_proba(X_test)

In [58]:
print('Model Accuracy: {:.2f}%'.format(average_precision_score(y_test, proba[:,1])*100))

Model Accuracy: 89.43%
